In [ ]:
%pip install smolagents "smolagents[litellm]" -qU
%pip install ddgs

# Building Autonomous Agents with smolagents

[🤗 smolagents](https://huggingface.co/blog/smolagents) is a library by Hugging Face that enables you to run powerful agents in a few lines of code. It is based on the concept of CodeAct Agents ([arXiv:2402.01030](https://arxiv.org/abs/2402.01030)), i.e. agents that write their actions in code. In a multi-step agent, at each step, the LLM can write an action, in the form of some calls to external tools. A common format for writing these actions is generally different shades of "writing actions as a JSON of tools names and arguments to use, which you then parse to know which tool to execute and with which arguments". To make it secure, it supports executing in sandboxed environments. Multiple research papers have shown that having the tool calling LLMs in code is much better.

In this example we will highlight how we can use a multi agent framework with tools to find the most downloaded model of a given task on the Hugging Face Hub. 

smolagents supports any LLM via LiteLLM integration. To use smolagents, we are going to leverage the LiteLLM implementation which allows us to use both Amazon Bedrock and Amazon SageMaker AI, according to our preferences:

In [ ]:
from smolagents.models import LiteLLMModel

# To use Amazon Bedrock:
model = LiteLLMModel(model_id=f"bedrock/us.amazon.nova-pro-v1:0", max_tokens=5*1024)
# To use Amazon SageMaker AI:
# model = LiteLLMModel(model_id=f"sagemaker_chat/YOUR-ENDPOINT-NAME-HERE", max_tokens=5*1024)

Agents with smolagents can be easily created via two classes, `CodeAgent` and `ToolCallingAgent`:

- **CodeAgent** generates executable Python code snippets, enabling complex logic and variable handling for tasks requiring multi-step operations or data manipulation; 
- **ToolCallingAgent** employs standardized JSON structures to define tool calls, aligning with common LLM provider implementations for simpler, structured interactions. 

According to `smolagents` developers, CodeAgents typically achieve better performance on complex benchmarks due to their code-first flexibility, while ToolCallingAgents suit systems prioritizing interoperability with existing tool-calling protocols. Both agent types share the same multi-step workflow but differ fundamentally in action representation and execution security considerations.

In [ ]:
from smolagents import CodeAgent

agent = CodeAgent(model=model, tools=[])
agent.run("What is 123*456?")

Let's extend functionalities of agents using tools. As you've learnt already in the foundations/tools section, tools are functions or query engines that the agent can use to perform specific tasks. One nice feature of smolagents is that it comes with tools pre-packaged (called **base tools**) which can be easily added with:

In [ ]:
agent = CodeAgent(model=model, add_base_tools=True, tools=[])
agent.run("Can you explain the origin of the 'Hello World' program?")

The available base tools are listed in [smolagents doc](https://smolagents.org/docs/agents-guided-tour/#4-toc-title):

- **DuckDuckGo web search**: performs a web search using DuckDuckGo browser.
- **Python code interpreter**: runs your LLM generated Python code in a secure environment. This tool will only be added to ToolCallingAgent if you initialize it with add_base_tools=True, since code-based agent can already natively execute Python code
- **Transcriber**: a speech-to-text pipeline built on Whisper-Turbo that transcribes an audio to text.

To create custom tools, we use `@tool` decorator which turns a function into a tool. For example, let's write a function that retrieve the most downloaded model from the HuggingFace Hub:

In [ ]:
from smolagents import tool
from huggingface_hub import list_models

@tool
def get_top_hf_model_from_task(task: str) -> str:
    """
    This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub.
    It returns the name of the checkpoint.

    Args:
        task: The task for which
    """
    most_downloaded_model = next(
        iter(list_models(filter=task, sort="downloads", direction=-1))
    )
    return most_downloaded_model.id

agent = CodeAgent(model=model, add_base_tools=True, tools=[get_top_hf_model_from_task])
agent.run("Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub?")

### Multi-Agents

You can easily build hierarchical multi-agent systems with smolagents. Here’s an example of making an agent that manages a specific web search agent using `DuckDuckGoSearchTool`:

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, ToolCallingAgent
import litellm

litellm.drop_params = True # Required to drop {"tool_choice": "auto"}, not supported by Bedrock/SageMaker

web_agent = ToolCallingAgent(
    name="web_search_agent",
    description="Runs web searches for you.",
    model=model, max_steps=3,
    tools=[DuckDuckGoSearchTool(max_results=5)],
)

manager_agent = CodeAgent(
    tools=[], model=model, managed_agents=[web_agent], max_steps=3
)
manager_agent.run("Who is the CEO of Amazon Web Services as of 2025?")

### Exercise: Build a multi-agent travel assistant

Your task is to create a multi-agent travel assistant based on the notions you've learnt above. Here are the steps that you need to go through, to give you a starting point:

1. Create one Supervisor agent, who is meant to act like a project manager for the team
2. Create a Travel Researcher agent, whose task is to research and compile interesting activities and attractions for a given location
3. Create a Travel Content Writer agent, whose task is to create engaging and informative content for the top 5 listicle
4. Create a Content Editor agent, whose task is to ensure the listicle is well-structured, engaging, and error-free